### Importing Files ###

In [24]:
import pandas as pd
import glob
import os

In [25]:
csv_files = glob.glob("inputs/*.csv")

dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    df['ETF'] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(df)

etfs_tall = pd.concat(dfs, ignore_index=True)
etfs_tall.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)
etfs_tall

,Date,Adj Close,ETF
0,2008-12-01,17.391161,AOR
1,2009-01-01,16.519028,AOR
2,2009-02-01,15.420570,AOR
3,2009-03-01,16.174002,AOR
4,2009-04-01,17.332678,AOR
...,...,...,...
1391,2023-12-01,17.706827,VTTVX
1392,2024-01-01,18.360001,VTTVX
1393,2024-02-01,18.700001,VTTVX
1394,2024-03-01,19.080000,VTTVX


In [26]:
etfs_tall['ret'] = etfs_tall.groupby('ETF')['Adj Close'].pct_change()

In [27]:
etfs_tall.drop(columns=['Adj Close'], inplace=True)

### Converting DF to Wide ###

In [28]:
etfs_wide = etfs_tall.pivot(index='Date', columns='ETF', values='ret')
etfs_wide

ETF,AOR,BND,SPY,VFWAX,VNQ,VTTVX
Date,,,,,,
1993-05-01,NaN,NaN,NaN,NaN,NaN,NaN
1993-06-01,NaN,NaN,-0.003456,NaN,NaN,NaN
1993-07-01,NaN,NaN,0.002198,NaN,NaN,NaN
1993-08-01,NaN,NaN,0.038328,NaN,NaN,NaN
1993-09-01,NaN,NaN,-0.013424,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-01-01,0.009720,0.004063,0.020045,-0.001646,-0.038880,0.036888
2024-02-01,0.022377,-0.016478,0.052187,0.032916,0.019788,0.018519
2024-03-01,0.023910,0.008589,0.029503,0.027073,0.010871,0.020321


### Keeping Rows with all ETFs present ###

In [29]:
etfs_full = etfs_wide.dropna()
etfs_full = etfs_full.reset_index(drop=False)
etfs_full

ETF,Date,AOR,BND,SPY,VFWAX,VNQ,VTTVX
0,2011-11-01,-0.004464,-0.000555,-0.004064,-0.028667,-0.038012,-0.008689
1,2011-12-01,-0.002563,0.004682,0.004080,-0.059793,0.037010,-0.022311
2,2012-01-01,0.042725,0.016028,0.053011,0.112972,0.075583,0.070097
3,2012-02-01,0.025124,-0.002022,0.043406,0.050892,-0.011508,0.032006
4,2012-03-01,0.008775,-0.004975,0.027660,-0.008312,0.043614,0.012860
...,...,...,...,...,...,...,...
145,2023-12-01,0.034825,0.032739,0.041433,0.034064,0.081121,0.005462
146,2024-01-01,0.009720,0.004063,0.020045,-0.001646,-0.038880,0.036888
147,2024-02-01,0.022377,-0.016478,0.052187,0.032916,0.019788,0.018519
148,2024-03-01,0.023910,0.008589,0.029503,0.027073,0.010871,0.020321


### Expanding Data (80 years) ###

In [33]:
wide_sim_life = etfs_full.sample(n=12*80, random_state=42, replace=True, ignore_index=True).reset_index()
wide_sim_life

ETF,index,Date,AOR,BND,SPY,VFWAX,VNQ,VTTVX
0,0,2020-05-01,0.039412,0.006927,0.047645,0.046869,0.017344,0.032919
1,1,2019-07-01,-0.010431,0.001508,0.020057,-0.007886,0.026829,0.003136
2,2,2013-01-01,0.035320,-0.004946,0.058645,0.045263,0.050005,0.055857
3,3,2020-09-01,-0.018639,-0.000937,-0.041282,-0.025792,-0.034128,-0.015797
4,4,2017-10-01,0.010484,-0.000210,0.028644,0.025357,-0.000541,0.014247
...,...,...,...,...,...,...,...,...
955,955,2011-11-01,-0.004464,-0.000555,-0.004064,-0.028667,-0.038012,-0.008689
956,956,2017-01-01,0.019919,0.006406,0.023894,0.048158,0.019013,0.044336
957,957,2016-05-01,0.005553,0.000039,0.017012,-0.010576,0.022485,0.004997
958,958,2021-05-01,0.015079,0.002272,0.006566,0.032626,0.008074,0.009765


In [38]:
# I want to convert wide_sim_life to a tall format
# I can use the melt function to do this
# I want the columns following 'Date' to be the 'ETF' column, the index column to be 'month' and the values to be 'ret'
# I can use the melt function to do this
etfs_full_tall = wide_sim_life.melt(id_vars='index', var_name='ETF', value_name='ret')

In [39]:
# I want to drop rows where 'ETF' is 'Date'
etfs_full_tall = etfs_full_tall[etfs_full_tall['ETF'] != 'Date']

In [40]:
etfs_full_tall

,index,ETF,ret
960,0,AOR,0.039412
961,1,AOR,-0.010431
962,2,AOR,0.03532
963,3,AOR,-0.018639
964,4,AOR,0.010484
...,...,...,...
6715,955,VTTVX,-0.008689
6716,956,VTTVX,0.044336
6717,957,VTTVX,0.004997
6718,958,VTTVX,0.009765


### Converting back to tall ###

In [41]:
etfs_full_tall = wide_sim_life.melt(id_vars='index', var_name='ETF', value_name='ret')
etfs_full_tall = etfs_full_tall[etfs_full_tall['ETF'] != 'Date']
etfs_full_tall.rename(columns={'index': 'month'}, inplace=True)
etfs_full_tall

,month,ETF,ret
960,0,AOR,0.039412
961,1,AOR,-0.010431
962,2,AOR,0.03532
963,3,AOR,-0.018639
964,4,AOR,0.010484
...,...,...,...
6715,955,VTTVX,-0.008689
6716,956,VTTVX,0.044336
6717,957,VTTVX,0.004997
6718,958,VTTVX,0.009765


### Converting Df to Wide ###

In [42]:
rets_wide = etfs_full_tall.pivot(index='month', columns='ETF', values='ret')
rets_wide

ETF,AOR,BND,SPY,VFWAX,VNQ,VTTVX
month,,,,,,
0,0.039412,0.006927,0.047645,0.046869,0.017344,0.032919
1,-0.010431,0.001508,0.020057,-0.007886,0.026829,0.003136
2,0.03532,-0.004946,0.058645,0.045263,0.050005,0.055857
3,-0.018639,-0.000937,-0.041282,-0.025792,-0.034128,-0.015797
4,0.010484,-0.00021,0.028644,0.025357,-0.000541,0.014247
...,...,...,...,...,...,...
955,-0.004464,-0.000555,-0.004064,-0.028667,-0.038012,-0.008689
956,0.019919,0.006406,0.023894,0.048158,0.019013,0.044336
957,0.005553,0.000039,0.017012,-0.010576,0.022485,0.004997


### Creating Additional ETFs ###

In [43]:
rets_wide['SPY_VFWAX'] = 0.5*rets_wide['SPY'] + 0.5 * rets_wide['VFWAX'] #domestic/international stock split
rets_wide['SPY_BND'] = 0.5*rets_wide['SPY'] + 0.5 * rets_wide['BND'] #stock/bond split
rets_wide['SPY_VNQ_BND'] = 0.5*rets_wide['SPY'] + 0.5 * rets_wide['VNQ'] #stock/bond/real estate split
rets_wide['SPY_VFWAX_BND'] = 0.4*rets_wide['SPY'] + 0.4 * rets_wide['VFWAX'] + 0.2 * rets_wide['BND'] #domestic/ international stock/bond split

In [44]:
rets_wide

ETF,AOR,BND,SPY,VFWAX,VNQ,VTTVX,SPY_VFWAX,SPY_BND,SPY_VNQ_BND,SPY_VFWAX_BND
month,,,,,,,,,,
0,0.039412,0.006927,0.047645,0.046869,0.017344,0.032919,0.047257,0.027286,0.032494,0.039191
1,-0.010431,0.001508,0.020057,-0.007886,0.026829,0.003136,0.006085,0.010782,0.023443,0.00517
2,0.03532,-0.004946,0.058645,0.045263,0.050005,0.055857,0.051954,0.026849,0.054325,0.040574
3,-0.018639,-0.000937,-0.041282,-0.025792,-0.034128,-0.015797,-0.033537,-0.021109,-0.037705,-0.027017
4,0.010484,-0.00021,0.028644,0.025357,-0.000541,0.014247,0.027,0.014217,0.014051,0.021558
...,...,...,...,...,...,...,...,...,...,...
955,-0.004464,-0.000555,-0.004064,-0.028667,-0.038012,-0.008689,-0.016366,-0.002309,-0.021038,-0.013203
956,0.019919,0.006406,0.023894,0.048158,0.019013,0.044336,0.036026,0.01515,0.021453,0.030102
957,0.005553,0.000039,0.017012,-0.010576,0.022485,0.004997,0.003218,0.008526,0.019748,0.002582


In [46]:
final_data = rets_wide.reset_index().melt(id_vars='month', var_name='ETF', value_name='ret')

In [50]:
final_data

,month,ETF,ret
0,0,AOR,0.039412
1,1,AOR,-0.010431
2,2,AOR,0.03532
3,3,AOR,-0.018639
4,4,AOR,0.010484
...,...,...,...
9595,955,SPY_VFWAX_BND,-0.013203
9596,956,SPY_VFWAX_BND,0.030102
9597,957,SPY_VFWAX_BND,0.002582
9598,958,SPY_VFWAX_BND,0.016131


In [51]:
final_data.to_csv('inputs/etf_returns.csv', index=False)